Title Section: Data Reading and Processing, Members: Guillermo Leon, Submission Date: 10/13/25

Introduction Section: This program aims to read various data types, identify and rectify issues in the 'damaged' files, which may include missing values, inconsistencies, and format-related problems. The clean information will then be combined from all three data types to create a cohesive dataset.

In [1]:
# importing required libraries
import pandas as pd

In [ ]:
import json

Handle damaged csv data: Within this block of code I decide to use a function .read_csv while skipping bad lines. This allowed me to access the file without running into errors, from this point I decided to rename the columns to their standardized names. I then cleaned and converted the data types for the key columns, removing commas and coercing invalid values to NaN. Finally, I identified all rows with any missing or invalid values across all columns and saved them into a seperate dataframe called bad_rows_df. 

In [ ]:
# Read the CSV file while skipping bad lines
df_csv = pd.read_csv('fortune500.csv', on_bad_lines='skip')  

# Creating a dictionary that will map the new column names
column_dictionary = { 
    '#Year': 'Year', 
    '(1)Rank': 'Rank', 
    '!Company': 'Company', 
    '(3)Revenue (in millions)': 'Revenue (in millions)', 
    'okjb)Profit (in millions)': 'Profit (in millions)'
} 

# Renaming columns in the DataFrame using names from the column_dictionary
df_csv.rename(columns=column_dictionary, inplace=True) 

# Making sure each column is in the correct data type by onverting data types with error coercion
# If conversion fails coercion will return NaN 
# Also removing any commas from Revenue and Profit to prevent any formating issues
df_csv['Year'] = pd.to_numeric(df_csv['Year'], errors='coerce').astype('Int64')  
df_csv['Rank'] = pd.to_numeric(df_csv['Rank'], errors='coerce').astype('Int64') 
df_csv['Revenue (in millions)'] = pd.to_numeric(df_csv['Revenue (in millions)'].str.replace(',', ''), errors='coerce') 
df_csv['Profit (in millions)'] = pd.to_numeric(df_csv['Profit (in millions)'].str.replace(',', ''), errors='coerce') 

# Check all columns for any missing or invalid data and store into bad_rows_df
bad_rows_df = df_csv[df_csv.isnull().any(axis=1)]

In [ ]:
# Printing the clean dataframe to see if any issues arised
df_csv

Handle damaged json Data: Within this block of code I read the JSON file line by line, ensuring each record had five fields by filling the missing keys with None, and loaded the data into a dataframe. I then cleaned the data by converting each column to the correct data type, handling any invalid entries using coerce, which would turn them into NaN. Finally I seprated the rows with missing or invalid values into df_json_bad and kept only the clean rows in df_json_clean, and then reseting the index for both dataframes.

In [ ]:
file_path = "lines.json"
data = []
title = ["Year", "Rank", "Company", "Revenue (in millions)", "Profit (in millions)"]

# Read lines.json line by line and parse JSON
with open(file_path, 'r') as file:
    for line in file:
        try:
            #parse the JSON string
            json_obj = json.loads(line)
            
            # Ensure all five expected keys (columns) exist, fill with None value if missing
            for key in title:
                if key not in json_obj:
                    json_obj[key] = None

            #append the processed JSON object to the list
            data.append(json_obj)
        except json.JSONDecodeError:
            continue
        
# Convert the list of dictionaries to a pandas DataFrame
df_json = pd.DataFrame(data)

# Convert types with coercion, assign values to NaN if errors occur
df_json['Year'] = pd.to_numeric(df_json['Year'], errors='coerce').astype('Int64')
df_json['Rank'] = pd.to_numeric(df_json['Rank'], errors='coerce').astype('Int64')
df_json['Revenue (in millions)'] = pd.to_numeric(df_json['Revenue (in millions)'], errors='coerce')
df_json['Profit (in millions)'] = pd.to_numeric(df_json['Profit (in millions)'], errors='coerce')
df_json['Company'] = df_json['Company'].astype('string').str.strip()

# Creating dictionary to plug into isnull() to find bad rows
critical_cols = ['Year', 'Rank', 'Company', 'Revenue (in millions)', 'Profit (in millions)']
df_json_bad = df_json[df_json[critical_cols].isnull().any(axis=1)].copy()

# Dropping all the bad rows from the df_json and reseting index
df_json_clean = df_json.drop(df_json_bad.index).reset_index(drop=True)
df_json_bad.reset_index(drop=True, inplace=True)


In [ ]:
#printing to clean dataframe to see if any issues arise
df_json_clean

Handle damaged txt file: This block of code reads an unstructured text file where each record is a block of key-values pairs seperated by blank lines, and it converts each block into a dictionary using the process_block() function. It then attempts to cast each value to its correct data type, if its succesfull then they will be stored in good_rows, if not they will be stored in bad_rows. Finally, both list are converted into a dataframe.

In [ ]:
file_path = 'unstructureddata.txt'

# Using the dictionary to map column names to their data types
expected_types = {
    "Year": int,
    "Rank": int,
    "Company": str,
    "Revenue (in millions)": float,
    "Profit (in millions)": float,
}

# Function to convert block of key-value lines into a dictionary
def process_block(block):
    entity = {}
    for item in block:
            key, value = item.split(':')
            entity[key] = value
    return entity

# creating lists to store values
good_rows = []
bad_rows = []
current_data = []

# read and process the file
with open(file_path, 'r') as file:
    for line in file:
        line = line.strip()

        #if line is blank, signals the end of a block
        if not line:
            if current_data:
                # Converts the list of strings into a dictionary
                dfcontent = process_block(current_data)
                # Try block will attempt to convert values to their mapped data types
                # If conversion succeeds then append the row to the good_rows list
                try:
                    typed_dfcontent = {
                        key: expected_types[key](dfcontent[key]) for key in expected_types
                    }
                    good_rows.append(typed_dfcontent)
                # If error occurs than append that row to the bad_rows list
                except Exception:
                    bad_rows.append(dfcontent)
                    current_data = []
        else:
            current_data.append(line)

# Handle the final dfcontent (if no blank line at end)
# checks if any remaining unprocessed dfcontent is still stored in current_data
if current_data:
    dfcontent = process_block(current_data)

    # Try block to attempt type conversion of the values if succeeds then place row in good_rows
    # If fails then place bad row in bad_rows
    try:
        typed_dfcontent = {
            key: expected_types[key](dfcontent[key]) for key in expected_types
        }
        good_rows.append(typed_dfcontent)
    except Exception:
        bad_rows.append(dfcontent)

# Convert to DataFrames
df_good = pd.DataFrame(good_rows)
df_bad = pd.DataFrame(bad_rows)


In [ ]:
# printing the clean dataframe to see if any errors arise
df_good

In [ ]:
# Combining all three clean dataframes into one dataframe and then printing it
combined_df = pd.concat([df_csv, df_json_clean, df_good], ignore_index=True)

In [ ]:
combined_df

Data Statistics: This code block calculates key summary statistics from a combined dataset, including the total count of good data records and the total count of bad data rows aggregated from the three sources. It identifies the number of unique companies and finds the companies with the highest revenue and profit between 1995 and 1998. Finally, it organizes all these results into a dictionary.

In [ ]:
# Aggregate data volume (good data count)
total_good_data = len(combined_df)

# Instances of missing data (bad data count)
missing_data_count = len(pd.concat([bad_rows_df, df_json_bad, df_bad], ignore_index=True, sort=False))

# Number of unique companies
unique_companies = combined_df['Company'].nunique()

# Company with highest revenue and highest profit between 1995 and 1998
filtered_df = combined_df[(combined_df['Year'] >= 1995) & (combined_df['Year'] <= 1998)]

max_revenue = filtered_df['Revenue (in millions)'].max()
highest_revenue_rows = filtered_df[filtered_df['Revenue (in millions)'] == max_revenue]
# Pick the first company with the highest revenue
company_highest_revenue = highest_revenue_rows.iloc[0]['Company']

max_profit = filtered_df['Profit (in millions)'].max()
# Now filter the rows where Profit equals max_profit (in case of ties)
highest_profit_rows = filtered_df[filtered_df['Profit (in millions)'] == max_profit]
# If you want just the first one (or you can handle multiple)
company_highest_profit = highest_profit_rows.iloc[0]['Company']

# Create a dictionary with all results
results_dict = {
    'Data Statistics:': [
        'Aggregate Data Volume (Good Data)',
        'Instances of Missing Data (Bad Data)',
        'Number of Unique Companies',
        'Company with Highest Revenue (1995-1998)',
        'Highest Revenue (1995-1998)',
        'Company with Highest Profit (1995-1998)',
        'Highest Profit (1995-1998)'
    ],
    '': [
        total_good_data,
        missing_data_count,
        unique_companies,
        company_highest_revenue,
        max_revenue,
        company_highest_profit,
        max_profit
    ]
}

# Convert dictionary to DataFrame and set Data Statistics as index
Results_Combine = pd.DataFrame(results_dict).set_index('Data Statistics:')

In [ ]:
print(Results_Combine)

In [ ]:
Results_Combine.to_csv("Results_Combine.csv")